<a href="https://colab.research.google.com/github/segomezz/Practica2-IA/blob/main/Pr%C3%A1ctica_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1) Importación y preparación del dataset

El dataset Lending Club contiene información financiera de préstamos otorgados entre 2007 y 2018. Cuenta con más de 2 millones de registros y múltiples variables tanto numéricas como categóricas. La variable objetivo seleccionada es loan_status, que representa el estado final del préstamo y posee más de 10 categorías, de las cuales seleccionamos al menos 4 para abordar el problema como clasificación multiclase. Este conjunto de datos es adecuado para aplicar técnicas de aprendizaje supervisado y no supervisado, ya que permite evaluar el desempeño de los modelos ante clases desbalanceadas y perfiles de riesgo.

In [16]:
import pandas as pd     #Importación de librerías necesarias
import numpy as np
import zipfile
from google.colab import drive
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from scipy.stats import zscore
import numpy as np
import pandas as pd
from sklearn.utils import resample
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
import seaborn as sns




In [ ]:
drive.mount('/content/drive')
# Cargar el archivo zip directamente
file_path = '/content/drive/MyDrive/2025-I/Introducción a la inteligencia Artificial/Practica 2/accepted_2007_to_2018Q4.csv.gz'

df_full = pd.read_csv(file_path, low_memory=False, compression='gzip')

# Tomar muestra aleatoria (conjunto representativo)
df = df_full.sample(n=20000, random_state=42).reset_index(drop=True)

# Verificación
print(f"Dataset cargado con {df.shape[0]} registros.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
# Ver las clases posibles de loan_status
df['loan_status'].value_counts()

NameError: name 'df' is not defined

In [7]:
# Filtrar las clases más frecuentes para convertir esto en clasificación multiclase
selected_statuses = ['Fully Paid', 'Charged Off', 'Late (31-120 days)', 'Default']
df = df[df['loan_status'].isin(selected_statuses)]

# Eliminar columnas con más del 50% de datos nulos
df = df.loc[:, df.isnull().mean() < 0.5]

# Seleccionar algunas variables numéricas y categóricas representativas
cat_vars = ['grade', 'home_ownership', 'verification_status', 'purpose', 'term']
num_vars = ['loan_amnt', 'int_rate', 'annual_inc', 'dti', 'open_acc', 'revol_util']

df = df[cat_vars + num_vars + ['loan_status']].dropna()

In [8]:
# Cambio de variables categóricas a numéricas con la introducción de una variable dummy
df_encoded = pd.get_dummies(df, columns=cat_vars)

In [9]:
# Estandarización de datos

scaler = StandardScaler()
num_scaled = scaler.fit_transform(df_encoded[num_vars])
df_encoded[num_vars] = num_scaled

In [10]:

# 1. Calcular los z-scores solo de las variables numéricas
z_scores = np.abs(zscore(df_encoded[num_vars]))

# 2. Filtrar el dataset sin outliers (z-score < 3 en TODAS las columnas numéricas)
df_no_outliers = df_encoded[(z_scores < 3).all(axis=1)]

# 3. Determinar cuántas filas queremos como "outliers" (5% del dataset limpio)
n_outliers = int(0.05 * len(df_no_outliers))

# 4. Obtener las filas que fueron eliminadas (los verdaderos outliers detectados)
df_all_outliers = df_encoded[~df_encoded.index.isin(df_no_outliers.index)]

# 5. Tomar una muestra aleatoria de outliers (solo 5%)
# Verifica cuántos outliers realmente tienes disponibles
available_outliers = df_all_outliers.shape[0]
n_outliers_final = min(n_outliers, available_outliers)

# Toma solo los que puedas (sin reemplazo)
df_outliers_5pct = df_all_outliers.sample(n=n_outliers_final, random_state=42)

# 6. Combinar los datos limpios + los outliers seleccionados
df_5pct_outliers = pd.concat([df_no_outliers, df_outliers_5pct])

In [11]:


# Dataset balanceado
min_class = df_no_outliers['loan_status'].value_counts().min()

dfs = []
for status in df_no_outliers['loan_status'].unique():
    df_class = df_no_outliers[df_no_outliers['loan_status'] == status]
    dfs.append(resample(df_class, replace=False, n_samples=min_class, random_state=42))

df_balanced = pd.concat(dfs)

# Dataset desbalanceado (sin modificación)
df_unbalanced = df_no_outliers.copy()

In [12]:
datasets = {}

# Base sin escalado: deshacer el escalado en una copia
df_unscaled = df_encoded.copy()
df_unscaled[num_vars] = df[num_vars]  # recuperamos los valores originales sin escalar

### CASOS 1-4: sin escalado
datasets['1'] = df_no_outliers.copy()                        # No escalado, sin outliers, desbalanceado
datasets['2'] = df_balanced.copy()                           # No escalado, sin outliers, balanceado
datasets['3'] = df_5pct_outliers.copy()                      # No escalado, con 5% outliers, desbalanceado
datasets['4'] = pd.concat([
    df_balanced,
    df_5pct_outliers[df_5pct_outliers['loan_status'].isin(df_balanced['loan_status'].unique())]
]).drop_duplicates().copy()  # Balanceado con 5% outliers

### CASOS 5-8: con escalado (ya aplicado en df_encoded y derivados)
datasets['5'] = df_no_outliers.copy()                        # Escalado, sin outliers, desbalanceado
datasets['6'] = df_balanced.copy()                           # Escalado, sin outliers, balanceado
datasets['7'] = df_5pct_outliers.copy()                      # Escalado, con 5% outliers, desbalanceado
datasets['8'] = pd.concat([
    df_balanced,
    df_5pct_outliers[df_5pct_outliers['loan_status'].isin(df_balanced['loan_status'].unique())]
]).drop_duplicates().copy()  # Escalado, balanceado, con 5% outliers

# 2) Aprendizaje supervisado

# 3) Aprendizaje no supervisado

## K-means

In [13]:
unsupervised_datasets = {} #Creación de dataset que se usarán en el entrenamiento

for i in range(1, 9):
    df_i = datasets[str(i)].drop(columns=['loan_status'])  # quitamos la variable objetivo
    unsupervised_datasets[str(i)] = df_i.copy()

In [15]:
def evaluate_kmeans(X, max_k=10):
    inertias = []
    silhouettes = []

    for k in range(2, max_k + 1):
        kmeans = KMeans(n_clusters=k, random_state=42)
        kmeans.fit(X)
        inertias.append(kmeans.inertia_)
        silhouettes.append(silhouette_score(X, kmeans.labels_))

    return inertias, silhouettes

In [17]:
X = unsupervised_datasets['1']

inertias, silhouettes = evaluate_kmeans(X)

plt.figure(figsize=(12,5))

plt.subplot(1, 2, 1)
plt.plot(range(2, 11), inertias, marker='o')
plt.title('Método del Codo (Inertia)')
plt.xlabel('k')
plt.ylabel('Inercia')

plt.subplot(1, 2, 2)
plt.plot(range(2, 11), silhouettes, marker='o', color='green')
plt.title('Silhouette Score')
plt.xlabel('k')
plt.ylabel('Silhouette')

plt.tight_layout()
plt.show()

KeyboardInterrupt: 